In [1]:
%load_ext autoreload
%autoreload 2
import os
from pathlib import Path
import numpy as np
import pandas as pd
import pyvista as pv
from tqdm.auto import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

In [2]:
root_dir = Path("/home/max/Desktop/python_projects/SimJeb/")
metadata_dir = root_dir / "SimJEB_metadata"
meshes_dir = root_dir / "SimJEB_volmesh"

In [17]:
features_grid = pv.read(metadata_dir / "grid_rare.vtk")

In [27]:
for ds in ["train", "test"]:
    df = pd.read_csv(metadata_dir / f"{ds}_bracket_metadata.csv")
    category_df = df["category"]
    features = []
    for id_ in tqdm(df["id"]):
        mesh = pv.read(meshes_dir / f"{str(id_)}.vtk")
        cell_ids = features_grid.find_containing_cell(mesh.points)
        cell_ids = cell_ids[cell_ids >= 0]
        features_tmp = np.zeros((features_grid.n_cells,))
        features_tmp[cell_ids] = 1
        features_tmp_dict = {f"feature/{i}": features_tmp[i] for i in range(len(features_tmp))}
        features_tmp_dict["id"] = id_
        features.append(features_tmp_dict)
    features_df = pd.DataFrame(features)
    category_df = pd.DataFrame(category_df).join(features_df)
    category_df.to_csv(metadata_dir / f"{ds}_bracket_category.csv")

  0%|          | 0/83 [00:00<?, ?it/s]

In [3]:
train_df = pd.read_csv(metadata_dir / "train_bracket_category.csv")

In [4]:
train_df.head()

,Unnamed: 0,category,feature/0,feature/1,feature/2,feature/3,feature/4,feature/5,feature/6,feature/7,...,feature/190,feature/191,feature/192,feature/193,feature/194,feature/195,feature/196,feature/197,feature/198,id
0,0,flat,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,241
1,1,block,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,281
2,2,block,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,170
3,3,flat,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,194
4,4,butterfly,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,497


In [4]:
le = preprocessing.LabelEncoder()
le.fit(train_df.category)
y_train = le.transform(train_df.category)

In [5]:
y_train.shape

(247,)

In [6]:
len(np.unique(y_train))

6

In [15]:
y_train

[autoreload of numpy.lib.ufunclike failed: Traceback (most recent call last):
  File "/home/max/miniconda3/envs/dev/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/home/max/miniconda3/envs/dev/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/home/max/miniconda3/envs/dev/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/home/max/miniconda3/envs/dev/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 289, in update_function
    setattr(old, name, getattr(new, name))
ValueError: _dispatcher() requires a code object with 0 free vars, not 1
]
[autoreload of numpy.matrixlib failed: Traceback (most recent call last):
  File "/home/max/miniconda3/envs/dev/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 257, in check
 

array([4, 2, 2, 4, 3, 4, 2, 0, 1, 2, 1, 1, 4, 4, 0, 4, 2, 1, 4, 1, 4, 1,
       2, 3, 4, 3, 2, 2, 4, 3, 0, 0, 1, 4, 4, 1, 2, 3, 2, 2, 2, 1, 4, 4,
       4, 4, 4, 4, 1, 4, 4, 2, 2, 4, 0, 4, 4, 4, 2, 2, 1, 0, 0, 4, 2, 2,
       4, 4, 3, 4, 2, 0, 2, 4, 4, 4, 1, 3, 0, 0, 3, 2, 4, 3, 2, 2, 1, 2,
       2, 5, 0, 4, 3, 5, 4, 4, 4, 0, 5, 0, 4, 2, 4, 4, 3, 4, 2, 0, 4, 4,
       2, 2, 4, 4, 1, 1, 4, 0, 0, 1, 2, 2, 2, 4, 1, 4, 2, 1, 3, 0, 3, 2,
       2, 3, 4, 2, 2, 2, 4, 1, 4, 4, 0, 2, 4, 4, 2, 2, 0, 0, 4, 2, 2, 2,
       4, 4, 2, 4, 2, 2, 4, 2, 2, 3, 3, 2, 3, 1, 1, 4, 5, 4, 0, 2, 2, 0,
       4, 2, 2, 2, 4, 2, 4, 1, 4, 2, 2, 1, 4, 3, 4, 4, 2, 2, 3, 4, 1, 4,
       3, 2, 0, 3, 4, 1, 3, 4, 0, 1, 2, 4, 4, 1, 2, 4, 4, 2, 4, 3, 0, 4,
       4, 1, 4, 2, 2, 1, 4, 2, 3, 3, 2, 4, 3, 4, 4, 4, 2, 4, 1, 3, 4, 3,
       4, 2, 2, 3, 4])

In [7]:
X_train = train_df.iloc[:,2:201].to_numpy()

In [8]:
X_train.shape

(247, 199)

In [9]:
clf = RandomForestClassifier()
estimator = clf.fit(X_train, y_train)

In [10]:
test_df = pd.read_csv(metadata_dir / "test_bracket_category.csv")

In [11]:
X_test = test_df.iloc[:,2:201].to_numpy()
y_test = le.transform(test_df.category)

In [12]:
y_test_pred = estimator.predict(X_test)

In [13]:
accuracy_score(y_test, y_test_pred)

0.7349397590361446